## Run all the required Dependencies

In [1]:
!pip install youtube-transcript-api
!pip install -q langchain-groq langchain-text-splitters langchain-core
!pip install fastapi[all] uvicorn python-multipart transformers pydantic
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307

## Run this to build restful api using fast api

In [35]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from youtube_transcript_api import YouTubeTranscriptApi
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser
import base64
from IPython.display import Image, display
import json
import ujson
import re
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    groq_api_key="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"
)

class HeadingSummary(BaseModel):
    heading: str = Field(description="A concise heading or title for a key point from the given query")
    paragraph: str = Field(description="Define and explain what the heading or title is about in simple terms that a 5-year-old could understand in 5 sentences")

class TextSummary(BaseModel):
    points: List[HeadingSummary] = Field(
        description="List of 10-15 heading-definition pairs with explanations extracted from the given query, covering the complete topic",
        min_items=10,
        max_items=15
    )

parser = PydanticOutputParser(pydantic_object=TextSummary)

prompt = PromptTemplate(
    template="""Answer the user query by providing 10-15 detailed key points, each with a concise heading and a simple explanation.

Format Instructions:
{format_instructions}

User Query:
{query}

Remember to:
1. Use simple language suitable for a 5-year-old
2. Cover the main aspects of the query
3. Provide clear and concise headings
4. Explain each heading in a short paragraph
5. Ensure the explanations are easy to understand
6. explain with an example

Your response:""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

class Flashcard(BaseModel):
    ques: str = Field(description="Question for the flashcard")
    options: List[str] = Field(description="Four options for the given question of the flashcard. Ensure one option is correct")
    correct_ans: str = Field(description="Correct answer to the question on the flashcard")
    category: str = Field(description="Category of the flashcard, whether the question is easy or hard")
    topic: str = Field(description="Sub-topic from where the question is derived")

class GrpFlashcard(BaseModel):
    flashes: List[Flashcard] = Field(description="Generate 10 flashcards")

parser_2 = PydanticOutputParser(pydantic_object=GrpFlashcard)

prompt_template = PromptTemplate(
    template=(
        "You are given a transcript of a YouTube lecture. Generate 5 easy questions "
        "which have direct answers from the transcript. Generate 5 hard questions "
        "which require deep conceptual thinking and are difficult to answer. "
        "Ensure each question has only one correct answer and directly relates to the material "
        "covered in the transcript. Based on the following context, generate a flashcard with a question, options, and answer:\n\n"
        "Context: {context}\n\nQuery: {query}\n\n{format_instructions}"
    ),
    input_variables=["context", "query"],
    partial_variables={"format_instructions": parser_2.get_format_instructions()}
)

def use_text_context(text, splitter):
    documents = splitter.create_documents([text])
    def inner_function(query):
        flashcards = []
        for doc in documents:
            context = doc.page_content
            input_data = {"context": context, "query": query}
            flashcards.append(input_data)
        return flashcards
    return inner_function

def create_rag_chain():
    return (
        RunnablePassthrough()
        | prompt_template
        | llm
        | parser_2
    )

def create_sum_rag_chain():
    return (
        RunnablePassthrough()
        | prompt__sum_template
        | llm
        | parser_sum
    )

def insert_line_breaks(text: str, n: int) -> str:
    words = text.split()
    return "\n".join([" ".join(words[i:i+n]) for i in range(0, len(words), n)])

def generate_flowchart(summaries: List[dict]):
    if not summaries:
        return "Error: No summaries provided", ""

    nodes = []
    for i, summary in enumerate(summaries):
        heading = summary.get('heading', f'Heading {i+1}')
        short_heading = insert_line_breaks(heading, 2)
        node_label = chr(65 + i)
        nodes.append(f"{node_label}[\"{short_heading}\"]")

    connections = []
    for i in range(len(summaries) - 1):
        short_summary = insert_line_breaks(summaries[i].get('summary', ''), 4)
        connections.append(f"{chr(65 + i)} -- {short_summary} --> {chr(65 + i + 1)}")

    flowchart = "flowchart LR\n"
    flowchart += "\n".join(nodes) + "\n"
    flowchart += "\n".join(connections)

    return mm(flowchart), flowchart

def mm(graph):
    graphbytes = graph.encode("ascii")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    url = "https://mermaid.ink/img/" + base64_string
    display(Image(url=url))
    print(url)
    return url

def generate_flashcards(text, query="Give a flashcard"):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=8000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )

    rag_chain = create_rag_chain()
    input_data_list = use_text_context(text, text_splitter)(query)

    flashcards = []
    for input_data in input_data_list:
        output = rag_chain.invoke(input_data)
        flashcards.append(output)

    return flashcards

class ChunkDetail(BaseModel):
    heading: str = Field(description="Heading of Text")
    summary: str = Field(description="Summary of Text")

class GrpDetails(BaseModel):
    grps: List[ChunkDetail] = Field(description="Generate summary", min_items=4, max_items=4)

parser_sum = PydanticOutputParser(pydantic_object=ChunkDetail)

prompt__sum_template = PromptTemplate(
    template="You are given a transcript of a YouTube lecture. Generate a heading and short summary of the text provided:\n\nContext: {context}\n\nQuery: {query}\n\n{format_instructions}",
    input_variables=["context", "query"],
    partial_variables={"format_instructions": parser_sum.get_format_instructions()}
)

@app.get('/{yt_id}')
async def root(yt_id: str):
    transcript = YouTubeTranscriptApi.get_transcript(yt_id)
    yt_transcript = " ".join([i["text"] for i in transcript])

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=8000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    texts = text_splitter.create_documents([yt_transcript])

    chain = prompt | llm

    summ_arr = []
    for text in texts:
        summ_arr.append(chain.invoke({"query": f"{text}"}).content)
    # return summ_arr
    summ_arr = summ_arr[0][summ_arr[0].find("{"):summ_arr[0].rfind("}")+1]

# Clean the string
    cleaned_string = re.sub(r'\\n', '', summ_arr)  # Remove escaped newlines
    cleaned_string = re.sub(r'\\(?!")', '', cleaned_string)  # Remove single backslashes except before quotes
    cleaned_string = re.sub(r'(?<!\\)"([^"]*)"', lambda m: '"%s"' % m.group(1).replace('"', '\\"'), cleaned_string)  # Escape unescaped quotes within strings

    # try:
    #     # Parse the JSON
    #     json_object = json.loads(cleaned_string)
    #     summ_arr = [json_object]
    #     # return summ_arr
    # except json.JSONDecodeError as e:
    #     print(f"JSON Decode Error: {e}")
    #     print(f"Problematic JSON string: {cleaned_string}")
    #     return []
    try:
      json_object = ujson.loads(cleaned_string)
      print("JSON parsed successfully")
      summ_arr = [json_object]
    except ValueError as e:
        print(f"JSON Decode Error: {e}")
        print(f"Problematic JSON string: {cleaned_string}")
        return []
    # return summ_arr

    # print(summ_arr)
    all_summ_points = []
    for summ in summ_arr:
        if summ is not None:
            points = summ.get("points", None)  # Get the list of
            if points is not None:
                all_summ_points.extend(points)

    return all_summ_points

@app.get('/quiz/{yt_id}')
async def get_flashcards(yt_id: str):
    transcript = YouTubeTranscriptApi.get_transcript(yt_id)
    yt_transcript = " ".join([i["text"] for i in transcript])

    flashcards = generate_flashcards(yt_transcript, "Give 5 flashcards with category easy and 5 with hard")

    all_questions = []
    for flashcard_group in flashcards:
        for flashcard in flashcard_group.flashes:
            all_questions.append({
                "question": flashcard.ques,
                "option_one": flashcard.options[0],
                "option_two": flashcard.options[1],
                "option_three": flashcard.options[2],
                "option_four": flashcard.options[3],
                "correct_ans": flashcard.correct_ans,
                "category": flashcard.category,
                "topic": flashcard.topic
            })

    return all_questions

@app.get('/flowchart/{yt_id}')
async def get_summ(yt_id: str):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(yt_id)
        yt_transcript = " ".join([i["text"] for i in transcript])

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=8000,
            chunk_overlap=200,
            length_function=len,
            is_separator_regex=False,
        )
        texts = text_splitter.create_documents([yt_transcript])

        def generate_summary(text, query="Give a summary"):
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=len(text)//3,
                chunk_overlap=200,
                length_function=len,
                is_separator_regex=False,
            )

            rag_chain = create_sum_rag_chain()
            input_data_list = use_text_context(text, text_splitter)(query)

            summaries = []
            for input_data in input_data_list:
                output = rag_chain.invoke(input_data)
                summaries.append(output)

            return summaries

        context_text = yt_transcript
        summaries = generate_summary(context_text)

        # Convert Pydantic models to dictionaries for the flowchart function
        summary_dicts = [{"heading": summary.heading, "summary": summary.summary} for summary in summaries]

        url, code = generate_flowchart(summary_dicts)
        all_summaries = [summary.summary for summary in summaries]
        return {"url": url, "description": all_summaries}

    except Exception as e:
        return f"Error: {str(e)}"

## Run the Server

- **Import libraries:**  
  - Essential for handling async operations, creating secure tunnels, and running the server.

- **Set ngrok token:**  
  - Required for authenticating and using ngrok's tunneling service.

- **Create ngrok tunnel:**  
  - Allows external access to your local server via a public URL.

- **Apply nest_asyncio:**  
  - Ensures compatibility with environments that have an existing event loop.

- **Run uvicorn server:**  
  - Launches your web application on the specified port for local development.

  ## The code below generates a public URL which is used as an API endpoint for the website 👇
  

In [37]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "2i30SHnxvrxUT4BKGJdXMAuvhAx_86wvKQ7ByxMTdhTpmyQ5r"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the uvicorn server
uvicorn.run(app, port=8000)


Public URL: https://ce97-35-197-64-18.ngrok-free.app


INFO:     Started server process [1338]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     103.25.231.104:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.25.231.104:0 - "GET /flowchart/l9AzO1FMgM8 HTTP/1.1" 200 OK
INFO:     103.25.231.104:0 - "GET /flowchart/l9AzO1FMgM8 HTTP/1.1" 200 OK
INFO:     103.25.231.104:0 - "GET /flowchart/l9AzO1FMgM8 HTTP/1.1" 200 OK
INFO:     103.25.231.104:0 - "GET /flowchart/l9AzO1FMgM8 HTTP/1.1" 200 OK
INFO:     103.25.231.104:0 - "GET /flowchart/l9AzO1FMgM8 HTTP/1.1" 200 OK
INFO:     103.25.231.104:0 - "GET /flowchart/l9AzO1FMgM8 HTTP/1.1" 200 OK


https://mermaid.ink/img/Zmxvd2NoYXJ0IExSCkFbIlRoZSBSaXNlCmFuZCBDcml0aWNpc21zCm9mIEphdmEiXQpCWyJUaGUgRGFyawpTaWRlIG9mCkphdmEiXQpDWyJHZXR0aW5nIFN0YXJ0ZWQKd2l0aCBKYXZhClByb2dyYW1taW5nIl0KRFsiVGhlIFJlYWxpdHkKb2YgUHJvZ3JhbW1pbmcKTGFuZ3VhZ2VzIl0KQSAtLSBKYXZhLCBhIHZlcmJvc2Ugb2JqZWN0LW9yaWVudGVkCmxhbmd1YWdlLCB3YXMgY3JlYXRlZCBpbgp0aGUgOTBzIGFuZCBoYXMKYmVlbiBydW5uaW5nIG9uIDMKYmlsbGlvbiBkZXZpY2VzIGZvciAyNQp5ZWFycywgZGVzcGl0ZSBpdHMgY3JpdGljaXNtcwphbmQgdGhlIGNyZWF0aW9uIG9mCmFsdGVybmF0aXZlIGxhbmd1YWdlcyB0byBpbXByb3ZlCnVwb24gaXQuIC0tPiBCCkIgLS0gSmF2YSwgYSBwcm9ncmFtbWluZyBsYW5ndWFnZQp0aGF0IGhhcyBpbnNwaXJlZCBtYW55Cm90aGVycywgaGFzIGl0cyBkb3duc2lkZXMsCmluY2x1ZGluZyBpdHMgY29tcGxleGl0eSwgdGhlCm5lZWQgZm9yIE9yYWNsZSBkYXRhYmFzZSwKYW5kIHRoZSBkaWZmaWN1bHRpZXMgb2YKaW5zdGFsbGF0aW9uIGFuZCBlcnJvciBoYW5kbGluZy4gLS0+IEMKQyAtLSBUaGlzIGxlY3R1cmUgY292ZXJzIHRoZQpiYXNpY3Mgb2Ygc2V0dGluZyB1cAphIEphdmEgZGV2ZWxvcG1lbnQgZW52aXJvbm1lbnQsCmluY2x1ZGluZyBpbnN0YWxsaW5nIEpESywgSlJFLAphbmQgSlZNLCBhbmQgd3JpdGluZwphICdIZWxsbyBXb3JsZCcg

https://mermaid.ink/img/Zmxvd2NoYXJ0IExSCkFbIlRoZSBSaXNlCmFuZCBDcml0aWNpc21zCm9mIEphdmEiXQpCWyJUaGUgRGFyawpTaWRlIG9mCkphdmEiXQpDWyJHZXR0aW5nIFN0YXJ0ZWQKd2l0aCBKYXZhClByb2dyYW1taW5nIl0KRFsiVGhlIFJlYWxpdHkKb2YgUHJvZ3JhbW1pbmcKTGFuZ3VhZ2VzIl0KQSAtLSBKYXZhLCBhIHZlcmJvc2Ugb2JqZWN0LW9yaWVudGVkCmxhbmd1YWdlLCB3YXMgY3JlYXRlZCBpbgp0aGUgOTBzIGFuZCBoYXMKYmVlbiBydW5uaW5nIG9uIDMKYmlsbGlvbiBkZXZpY2VzIGZvciAyNQp5ZWFycywgZGVzcGl0ZSBpdHMgY3JpdGljaXNtcwphbmQgdGhlIGNyZWF0aW9uIG9mCmFsdGVybmF0aXZlIGxhbmd1YWdlcyB0byBpbXByb3ZlCnVwb24gaXQuIC0tPiBCCkIgLS0gSmF2YSwgYSBwcm9ncmFtbWluZyBsYW5ndWFnZQp0aGF0IGhhcyBpbnNwaXJlZCBtYW55Cm90aGVycywgaGFzIGl0cyBkb3duc2lkZXMsCmluY2x1ZGluZyBpdHMgY29tcGxleGl0eSwgdGhlCm5lZWQgZm9yIE9yYWNsZSBkYXRhYmFzZSwKYW5kIHRoZSBkaWZmaWN1bHRpZXMgb2YKaW5zdGFsbGF0aW9uIGFuZCBlcnJvciBoYW5kbGluZy4gLS0+IEMKQyAtLSBUaGlzIGxlY3R1cmUgY292ZXJzIHRoZQpiYXNpY3Mgb2Ygc2V0dGluZyB1cAphIEphdmEgZGV2ZWxvcG1lbnQgZW52aXJvbm1lbnQsCmluY2x1ZGluZyBpbnN0YWxsaW5nIEpESywgSlJFLAphbmQgSlZNLCBhbmQgd3JpdGluZwphICdIZWxsbyBXb3JsZCcg

...

https://mermaid.ink/img/Zmxvd2NoYXJ0IExSCkFbIkludHJvZHVjdGlvbiB0bwpMdWEiXQpCWyJJbnRyb2R1Y3Rpb24gdG8KTHVhIl0KQ1siTHVhIExhbmd1YWdlCkZlYXR1cmVzIl0KRFsiTHVhIGluCjEwMCBTZWNvbmRzIl0KQSAtLSBMdWEgaXMgYSBmYXN0LAptdWx0aS1wYXJhZGlnbSBzY3JpcHRpbmcgbGFuZ3VhZ2UgdGhhdAppcyBlYXN5IHRvIGxlYXJuLApwb3J0YWJsZSwgYW5kIHdpZGVseSB1c2VkCmluIGFwcGxpY2F0aW9ucyBsaWtlIFdvcmxkCm9mIFdhcmNyYWZ0IGFuZCBSb2Jsb3gKZHVlIHRvIGl0cyBzcGVlZAphbmQgZW1iZWRkYWJpbGl0eS4gLS0+IEIKQiAtLSBMdWEgaXMgYSBkeW5hbWljCmxhbmd1YWdlIHdpdGggYSBzaW5nbGUKZGF0YSBzdHJ1Y3R1cmluZyBtZWNoYW5pc20gY2FsbGVkCmEgdGFibGUsIHN1cHBvcnRpbmcgY29sbGFib3JhdGl2ZQptdWx0aXRhc2tpbmcgYW5kIGEgbWluaW1hbApzdGFuZGFyZCBsaWJyYXJ5LCBidXQgd2l0aAphIGxhcmdlIGVjb3N5c3RlbSBvZgpwYWNrYWdlcy4gLS0+IEMKQyAtLSBUaGlzIHRleHQgZGVzY3JpYmVzIHRoZQpmZWF0dXJlcyBvZiB0aGUgTHVhCnByb2dyYW1taW5nIGxhbmd1YWdlLCBpbmNsdWRpbmcgaXRzCnN1cHBvcnQgZm9yIGZ1bmN0aW9uYWwgcHJvZ3JhbW1pbmcsCmRhdGEgc3RydWN0dXJpbmcgd2l0aCB0YWJsZXMsCmFuZCBjb25jdXJyZW5jeSB3aXRoIGNvcm91dGluZXMuIC0tPiBE
INFO:     103.25.231.104:0 - "GET /flow

https://mermaid.ink/img/Zmxvd2NoYXJ0IExSCkFbIkludHJvZHVjdGlvbiB0bwpMdWEiXQpCWyJJbnRyb2R1Y3Rpb24gdG8KTHVhIl0KQ1siTHVhIExhbmd1YWdlCkZlYXR1cmVzIl0KRFsiTHVhIGluCjEwMCBTZWNvbmRzIl0KQSAtLSBMdWEgaXMgYSBmYXN0LAptdWx0aS1wYXJhZGlnbSBzY3JpcHRpbmcgbGFuZ3VhZ2UgdGhhdAppcyBlYXN5IHRvIGxlYXJuLApwb3J0YWJsZSwgYW5kIHdpZGVseSB1c2VkCmluIGFwcGxpY2F0aW9ucyBsaWtlIFdvcmxkCm9mIFdhcmNyYWZ0IGFuZCBSb2Jsb3gKZHVlIHRvIGl0cyBzcGVlZAphbmQgZW1iZWRkYWJpbGl0eS4gLS0+IEIKQiAtLSBMdWEgaXMgYSBkeW5hbWljCmxhbmd1YWdlIHdpdGggYSBzaW5nbGUKZGF0YSBzdHJ1Y3R1cmluZyBtZWNoYW5pc20gY2FsbGVkCmEgdGFibGUsIHN1cHBvcnRpbmcgY29sbGFib3JhdGl2ZQptdWx0aXRhc2tpbmcgYW5kIGEgbWluaW1hbApzdGFuZGFyZCBsaWJyYXJ5LCBidXQgd2l0aAphIGxhcmdlIGVjb3N5c3RlbSBvZgpwYWNrYWdlcy4gLS0+IEMKQyAtLSBUaGlzIHRleHQgZGVzY3JpYmVzIHRoZQpmZWF0dXJlcyBvZiB0aGUgTHVhCnByb2dyYW1taW5nIGxhbmd1YWdlLCBpbmNsdWRpbmcgaXRzCnN1cHBvcnQgZm9yIGZ1bmN0aW9uYWwgcHJvZ3JhbW1pbmcsCmFzc29jaWF0aXZlIGFycmF5cywgYW5kIGNvcm91dGluZXMuIC0tPiBE
INFO:     103.25.231.104:0 - "GET /flowchart/jUuqBZwwkQw HTTP/1.1" 200 OK
J

https://mermaid.ink/img/Zmxvd2NoYXJ0IExSCkFbIkludHJvZHVjdGlvbiB0bwpMdWEiXQpCWyJJbnRyb2R1Y3Rpb24gdG8KTHVhIl0KQ1siTHVhIExhbmd1YWdlCkZlYXR1cmVzIl0KRFsiTHVhIGluCjEwMCBTZWNvbmRzIl0KQSAtLSBMdWEgaXMgYSBmYXN0LAptdWx0aS1wYXJhZGlnbSBzY3JpcHRpbmcgbGFuZ3VhZ2UgdGhhdAppcyBlYXN5IHRvIGxlYXJuLApwb3J0YWJsZSwgYW5kIHdpZGVseSB1c2VkCmluIGFwcGxpY2F0aW9ucyBsaWtlIFdvcmxkCm9mIFdhcmNyYWZ0IGFuZCBSb2Jsb3gKZHVlIHRvIGl0cyBzcGVlZAphbmQgZW1iZWRkYWJpbGl0eS4gLS0+IEIKQiAtLSBMdWEgaXMgYSBkeW5hbWljCmxhbmd1YWdlIHdpdGggYSBzaW5nbGUKZGF0YSBzdHJ1Y3R1cmluZyBtZWNoYW5pc20gY2FsbGVkCmEgdGFibGUsIHN1cHBvcnRpbmcgY29sbGFib3JhdGl2ZQptdWx0aXRhc2tpbmcgYW5kIGEgbWluaW1hbApzdGFuZGFyZCBsaWJyYXJ5LCBidXQgd2l0aAphIGxhcmdlIGVjb3N5c3RlbSBvZgpwYWNrYWdlcy4gLS0+IEMKQyAtLSBUaGlzIHRleHQgZGVzY3JpYmVzIHRoZQpmZWF0dXJlcyBvZiB0aGUgTHVhCnByb2dyYW1taW5nIGxhbmd1YWdlLCBpbmNsdWRpbmcgaXRzCnN1cHBvcnQgZm9yIGZ1bmN0aW9uYWwgcHJvZ3JhbW1pbmcsCmRhdGEgc3RydWN0dXJpbmcgd2l0aCB0YWJsZXMsCmFuZCBjb25jdXJyZW5jeSB3aXRoIGNvcm91dGluZXMuIC0tPiBE
INFO:     103.25.231.104:0 - "GET /flow

https://mermaid.ink/img/Zmxvd2NoYXJ0IExSCkFbIkludHJvZHVjdGlvbiB0bwpMdWEiXQpCWyJJbnRyb2R1Y3Rpb24gdG8KTHVhIl0KQ1siTHVhIExhbmd1YWdlCkZlYXR1cmVzIl0KRFsiTHVhIGluCjEwMCBTZWNvbmRzIl0KQSAtLSBMdWEgaXMgYSBmYXN0LAptdWx0aS1wYXJhZGlnbSBzY3JpcHRpbmcgbGFuZ3VhZ2UgdGhhdAppcyBlYXN5IHRvIGxlYXJuLApwb3J0YWJsZSwgYW5kIHdpZGVseSB1c2VkCmluIGFwcGxpY2F0aW9ucyBsaWtlIFdvcmxkCm9mIFdhcmNyYWZ0IGFuZCBSb2Jsb3gKZHVlIHRvIGl0cyBzcGVlZAphbmQgZW1iZWRkYWJpbGl0eS4gLS0+IEIKQiAtLSBMdWEgaXMgYSBkeW5hbWljCmxhbmd1YWdlIHdpdGggYSBzaW5nbGUKZGF0YSBzdHJ1Y3R1cmluZyBtZWNoYW5pc20gY2FsbGVkCmEgdGFibGUsIHN1cHBvcnRpbmcgY29sbGFib3JhdGl2ZQptdWx0aXRhc2tpbmcgd2l0aCBjby1yb3V0aW5lcywgYW5kCmEgbWluaW1hbCBzdGFuZGFyZCBsaWJyYXJ5CndpdGggYSBsYXJnZSBlY29zeXN0ZW0Kb2YgcGFja2FnZXMuIC0tPiBDCkMgLS0gVGhpcyB0ZXh0IGRlc2NyaWJlcyB0aGUKZmVhdHVyZXMgb2YgdGhlIEx1YQpwcm9ncmFtbWluZyBsYW5ndWFnZSwgaW5jbHVkaW5nIGl0cwpzdXBwb3J0IGZvciBmdW5jdGlvbmFsIHByb2dyYW1taW5nLApkYXRhIHN0cnVjdHVyaW5nIHdpdGggdGFibGVzLAphbmQgY29uY3VycmVuY3kgd2l0aCBjb3JvdXRpbmVzLiAtLT4gRA==
INFO:     103.25.23

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1338]
